# Initial Flyback Calculations (LTSpice)

In [1]:
import math

In [2]:
# Input Variables
v_in_min = 7 # Input voltage (V)
v_in = 12 # Input voltage (V)
v_in_max = 25 # Input voltage (V)
v_out = 2000 # Output voltage (V)
i_out = 0.005 # Output current (A)
v_out_ripple = 0.05 # Percentage
np = 10 # Transformer primary turns 
ns = 1350 # Transformer secondary turns
lp = 19*10**(-6) # Transformer primary inductance (datasheet) (Henry's)
ls = 1 # Transformer secondary (datasheet) (Herny's)
lr_p = 0.120 # Transformer primary resistance (Ohm's)
lr_s = 8100 # Transformer secondary resistance (Ohm's)
i_primary_lim = 0.5 # Primary transformer winding current limit (A)

# Calculations
nps = np/ns # primary-to-secondary turns ratio

In [3]:
# Calculate the secondary inductance ratio given the turns ratio to properly model the transformer
ls_calc = lp/(np/ns)**2
print("Secondary Inductance Ls: %.3f H" % ls_calc) 

Secondary Inductance Ls: 0.346 H


Why doesn't Ls_calc match the datasheet?... LTSpice initial simulation stalls out with 0V output after a quick spike with 1H secondary inductance...

# LT3748 Calculations

In [11]:
# Input Variable
v_df = 1.1 # Forward voltage rating (V)
v_dr = 1000 # Reverse voltage rating (V)
rref = 6040 # LT3748 datasheet given (Ohm's)
v_tc = 0.55 # LT3748 datasheet given (V)
r_uvlo_r1 = 332000
r_uvlo_r2 = 41200


# Calculations
df_cnt = math.ceil((v_out*2)/v_dr) # 50% margin
# Calculate diode leakage safety resistors (http://engineering.electrical-equipment.org/electrical-distribution/series-and-parallel-connected-diodes.html)
v_bg_typ = 1.223 # LT3748 internal bandgap voltage referernce typical voltage (V)
v_df_total = v_df*df_cnt

In [12]:
# Calculate UVLO characteristics and component values
uvlo_rising = (1.223*(r_uvlo_r1+r_uvlo_r2))/r_uvlo_r2+0.0000024*r_uvlo_r1
uvlo_falling = (1.223*(r_uvlo_r1+r_uvlo_r2)/r_uvlo_r2)
print(uvlo_rising, uvlo_falling)


11.875042718446602 11.078242718446603


In [13]:
# LT3748 Feedback resistor
r_fb = (rref*(nps)*((v_out+v_df_total)+v_tc))/(v_bg_typ)

print("R_FB = %d Ohms" % r_fb)

R_FB = 73346 Ohms


In [14]:
# Calculate revised R_FB with actual measurement due to tolerances
v_out_measured = 1800
r_fb_revised = v_out/v_out_measured*r_fb
print("R_FB(Revised) = %d Ohms" % r_fb_revised)

R_FB(Revised) = 81496 Ohms


In [15]:
# Calculate R_TC (May skip this since my diodes will NOT be low forward drop, likely...)
# Use the iteration test in datashet for value(s)
v_out_1 = 1800
v_out_2 = 1805
temp1 = 25
temp2 = 30

r_tc = (r_fb/nps)*(0.00185/((v_out_2-v_out_1)/(temp2-temp1)))
print("R_TC = %.5f Ohms" % r_tc)

R_TC = 18318.32486 Ohms


In [16]:
# Calulate minimum primary inductance due to minimum off time
r_sense = 0.005 # Ohms
t_settle_min = 400*10**(-9)
v_sense_min = 0.015 # Volts

l_pri_min_toff = ((v_out+v_df_total)*r_sense*t_settle_min*nps)/(v_sense_min)
print("L_Primary(min_toff) = %.2f uH" % (l_pri_min_toff*1*10**(6)))

L_Primary(min_toff) = 1.98 uH


In [17]:
# Calculation minimum primary inductance due to input voltage (due to gate note timing)
t_on_min = 250*10**-9
# need to add v_in_max
l_pri_min_vin_min = (v_in_min*r_sense*t_on_min)/(v_sense_min)
l_pri_min_vin = (v_in*r_sense*t_on_min)/(v_sense_min)
l_pri_min_vin_max = (v_in_max*r_sense*t_on_min)/(v_sense_min)
print("L_Primary(min_vin) = %.2f , %.2f, %.2f uH" % ((l_pri_min_vin_min*1*10**(6)), (l_pri_min_vin*1*10**(6)), (l_pri_min_vin_max*1*10**(6))))

L_Primary(min_vin) = 0.58 , 1.00, 2.08 uH


In [38]:
# Calculate the minimum primary inductance due to minimum full-load operating frequency
# need to add v_in_min
#f_sw_min = 1/(t_on_min + t_off_min)
#l_pri_min_fmin = (v_in*(v_out+v_df_total)*nps)/(f_sw_min*i_lim*((v_out+v_df_total)*nps+v_in))

In [62]:
v_ds_min_min = v_in_min+v_out*nps
v_ds_min = v_in+v_out*nps
v_ds_min_max = v_in_max+v_out*nps
print("Minimum Vds rating = %.2f %.2f, %.2f V" % (v_ds_min_min, v_ds_min, v_ds_min_max))

Minimum Vds rating = 21.81 26.81, 39.81 V


In [64]:
v_df_r_min_min = v_out+(v_in_min/nps)
v_df_r_min = v_out+(v_in/nps)
v_df_r_min_max = v_out+(v_in_max/nps)
print("Minimum Vdf rating = %.2f %.2f, %.2f V" % (v_df_r_min_min, v_df_r_min, v_df_r_min_max))

Minimum Vdf rating = 2945.00 3620.00, 5375.00 V


In [65]:
# Calculate duty cycle
dc_min = ((v_out+v_df_total)*nps)/(v_in_min+(v_out+v_df_total)*nps)
dc = ((v_out+v_df_total)*nps)/(v_in+(v_out+v_df_total)*nps)
dc_max = ((v_out+v_df_total)*nps)/(v_in_max+(v_out+v_df_total)*nps)
print("Duty Cycle (D) = %.2f, %.2f, %.2f" % (dc_min, dc, dc_max))

Duty Cycle (D) = 0.68, 0.55, 0.37


In [67]:
# Calculate output diode RMS current
i_df_rms_min =((((i_primary_lim*nps)**2)*(1-dc_min))/(3))**(1/2)
i_df_rms =((((i_primary_lim*nps)**2)*(1-dc))/(3))**(1/2)
i_df_rms_max =((((i_primary_lim*nps)**2)*(1-dc_max))/(3))**(1/2)

print("i_df_rms = %.5f, %.5f, %.5f" % (i_df_rms_min, i_df_rms, i_df_rms_max))

i_df_rms = 0.00121, 0.00143, 0.00169


In [69]:
p_out_max_min = i_df_rms_min*v_out
p_out_max = i_df_rms*v_out
p_out_max_max = i_df_rms_max*v_out
print("P_out_max = %.1f, %.1f, %.1f W" % (p_out_max_min, p_out_max, p_out_max_max))

P_out_max = 2.4, 2.9, 3.4 W


In [70]:
c_out = (lp*i_primary_lim**2)/(2*(v_out*v_out_ripple*v_out))
print("C_output = %.2f pF" % (c_out*1*10**12))

C_output = 11.87 pF


# PCB Calculation

In [71]:
# Minimum spacings
mils_per_mm = 39.3700787
vclearance_ipc_b2_min = 0.25 # Minimum >500V base space IPC-2221 B2 (External conductors uncoated)vclearance_ipc_b2_vpermm = # IPC-2221 B2 (External conductors uncoated)
vclearance_ipc_b2_vpermm = 0.005 # mm/volt above 500V IPC-2221 B2 (External conductors uncoated)vclearance_ipc_b2_vpermm = # IPC-2221 B2 (External conductors uncoated)
vclearance_ipc_b4_min = 0.8 # Minimum >500V base space IPC-2221 B4 (external conductors with polymer coating)
vclearance_ipc_b4_vpermm = 0.00305 # mm/volt above 500V IPC-2221 B4 (external conductors with polymer coating)

mm_b2_min = vclearance_ipc_b2_min + ((v_out+v_out*v_out_ripple)*vclearance_ipc_b2_vpermm)
mm_b4_min = vclearance_ipc_b4_min + ((v_out+v_out*v_out_ripple)*vclearance_ipc_b4_vpermm)

print("Minimum exposed V_out trace spacing: %.1f mils" % (mm_b2_min*mils_per_mm))
print("Minimum silkmask V_out trace spacing: %.1f mils" % (mm_b4_min*mils_per_mm))

Minimum exposed V_out trace spacing: 423.2 mils
Minimum silkmask V_out trace spacing: 283.7 mils
